In [ ]:
class FeatureEngineer:
    def __init__(self, data):
        self.data = data.copy()
        self.finished_features = []
        
    def run_pipeline(self):
        self.core_price_features()
        self.math_rule_features()
        self.momentum_features()
        self.volatility_features()
        self.volume_features()
        self.derived_features()
        self.lagged_features()
        self.target_variable()
        return self.data
    
    def _apply_by_ticker(self, func): #splits data by ticker for feature calculation and reapplies
        return self.data.groupby(level='ticker', group_keys=False).apply(func)
    
    #create core price features for both models
    def core_price_features(self):
        def calc(df):
            close, high, low, open_ = df['close'], df['high'], df['low'], df['open']
            
            # Returns
            df['log_return'] = np.log(close / close.shift(1))
            df['overnight_return'] = np.log(open_ / close.shift(1))
            df['intraday_return'] = np.log(close / open_)
            # Volatility
            df['volatility_20d'] = df['log_return'].rolling(20).std() * np.sqrt(252)
            df['atr_14'] = talib.ATR(high, low, close, timeperiod=14)
            # SMAs
            sma10 = talib.SMA(close, timeperiod=10)
            sma20 = talib.SMA(close, timeperiod=20)
            sma50 = talib.SMA(close, timeperiod=50)
            #ratios
            df['price_sma20_ratio'] = close / sma20
            df['price_sma50_ratio'] = close / sma50
            df['sma10_sma20_ratio'] = sma10 / sma20
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['log_return', 'overnight_return', 'intraday_return', 
                            'volatility_20d', 'atr_14', 'price_sma20_ratio', 
                            'price_sma50_ratio', 'sma10_sma20_ratio']
    
    # Rule-Based Binary Features 
    
    def math_rule_features(self):
        """15 binary features for rule-based model"""
        def calc(df):
            close, high, low, volume = df['close'], df['high'], df['low'], df['volume']
            
            # SMAs
            sma10 = talib.SMA(close, timeperiod=10)
            sma20 = talib.SMA(close, timeperiod=20)
            sma50 = talib.SMA(close, timeperiod=50)
            sma200 = talib.SMA(close, timeperiod=200)
            
            # Trends
            df['golden_cross'] = (sma50 > sma200).astype(int)
            df['short_uptrend'] = (sma10 > sma20).astype(int)
            df['price_above_sma20'] = (close > sma20).astype(int)
            df['price_above_sma50'] = (close > sma50).astype(int)
            
            # Momentum
            rsi = talib.RSI(close, timeperiod=14)
            macd, signal, _ = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
            stoch_k, _ = talib.STOCH(high, low, close, fastk_period=14, slowk_period=3, slowd_period=3)
            roc = talib.ROC(close, timeperiod=10)
            
            df['rsi_oversold'] = (rsi < 30).astype(int)
            df['rsi_overbought'] = (rsi > 70).astype(int)
            df['macd_bullish'] = (macd > signal).astype(int)
            df['roc_positive'] = (roc > 0).astype(int)
            df['stoch_oversold'] = (stoch_k < 20).astype(int)
            
            # Volatility/Reversion 
            upper, _, lower = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2)
            bb_pos = (close - lower) / (upper - lower)
            vol_20d = df['log_return'].rolling(20).std() * np.sqrt(252)
            vol_75pct = vol_20d.expanding().quantile(0.75)
            
            df['bb_oversold'] = (bb_pos < 0.2).astype(int)
            df['bb_overbought'] = (bb_pos > 0.8).astype(int)
            df['high_volatility'] = (vol_20d > vol_75pct).astype(int)
            
            # Volume 
            vol_sma20 = talib.SMA(volume, timeperiod=20)
            vol_ratio = volume / vol_sma20
            price_up = close > close.shift(1)
            price_down = close < close.shift(1)
            
            df['volume_spike'] = (vol_ratio > 1.5).astype(int)
            df['volume_confirmation'] = (price_up & (vol_ratio > 1)).astype(int)
            df['volume_divergence'] = (price_down & (vol_ratio > 1)).astype(int)
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['golden_cross', 'short_uptrend', 'price_above_sma20', 'price_above_sma50',
                            'rsi_oversold', 'rsi_overbought', 'macd_bullish', 'roc_positive', 'stoch_oversold',
                            'bb_oversold', 'bb_overbought', 'high_volatility',
                            'volume_spike', 'volume_confirmation', 'volume_divergence']
    
    # XGBoost features

    def momentum_features(self):
        """5 momentum oscillators"""
        def calc(df):
            close, high, low = df['close'], df['high'], df['low']
            
            df['rsi_14'] = talib.RSI(close, timeperiod=14)
            macd, signal, hist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
            df['macd_histogram'] = hist
            df['stoch_k'], _ = talib.STOCH(high, low, close, fastk_period=14, slowk_period=3, slowd_period=3)
            df['williams_r'] = talib.WILLR(high, low, close, timeperiod=14)
            df['roc_10'] = talib.ROC(close, timeperiod=10)
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['rsi_14', 'macd_histogram', 'stoch_k', 'williams_r', 'roc_10']
    
    def volatility_features(self):
        """4 volatility & range features"""
        def calc(df):
            close, high, low = df['close'], df['high'], df['low']
            
            # Bollinger Bands
            upper, middle, lower = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2)
            df['bb_position'] = (close - lower) / (upper - lower)
            df['bb_width'] = (upper - lower) / middle
            
            # Parkinson volatility (high-low based)
            df['parkinson_vol'] = np.sqrt((1 / (4 * np.log(2))) * (np.log(high / low) ** 2)).rolling(20).mean()
            
            # Short-term volatility
            df['returns_std_5d'] = df['log_return'].rolling(5).std()
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['bb_position', 'bb_width', 'parkinson_vol', 'returns_std_5d']
    
    def volume_features(self):
        """3 volume indicators"""
        def calc(df):
            close, volume = df['close'], df['volume']
            
            vol_sma20 = talib.SMA(volume, timeperiod=20)
            df['volume_ratio'] = volume / vol_sma20
            df['obv'] = talib.OBV(close, volume)
            df['volume_zscore'] = (volume - volume.rolling(20).mean()) / volume.rolling(20).std()
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['volume_ratio', 'obv', 'volume_zscore']
    
    
    # Derived and time features 
    
    def derived_features(self):
        """6 derived interaction features"""
        def calc(df):
            df['rsi_slope_3d'] = df['rsi_14'].diff(3)
            df['macd_slope_3d'] = df['macd_histogram'].diff(3)
            df['rsi_volume_interaction'] = df['rsi_14'] * df['volume_ratio']
            df['momentum_vol_interaction'] = df['macd_histogram'] * df['atr_14']
            df['returns_skew_20d'] = df['log_return'].rolling(20).skew()
            df['returns_kurt_20d'] = df['log_return'].rolling(20).kurt()
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += ['rsi_slope_3d', 'macd_slope_3d', 'rsi_volume_interaction','momentum_vol_interaction', 'returns_skew_20d', 'returns_kurt_20d']
    
    def lagged_features(self):
        """6 lagged features (1-day lag to avoid look-ahead bias)"""
        lag_cols = ['log_return', 'rsi_14', 'volume_ratio', 'macd_histogram', 'bb_position', 'atr_14']
        
        def calc(df):
            for col in lag_cols:
                df[f'{col}_lag1'] = df[col].shift(1)
            return df
        
        self.data = self._apply_by_ticker(calc)
        self.finished_features += [f'{col}_lag1' for col in lag_cols]
    
    def target_variable(self):
        """Target: next day's return (shifted to avoid look-ahead bias)"""
        def calc(df):
            df['target'] = df['log_return'].shift(-1)  # Predict tomorrow's return
            return df
        self.data = self._apply_by_ticker(calc)

# Run the feature engineering pipeline
fe = FeatureEngineer(data)
features_data = fe.run_pipeline()

# Preview

features_data.groupby(level='ticker').head(3)